In [1]:
import numpy as np
import cv2
import os
import time
import csv
import shutil  # Import shutil for file and directory operations


In [2]:
import os
import cv2
import time

def capture_images(output_folder, num_images=10):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open default camera (camera index 0)
    cap = cv2.VideoCapture(0)

    # Ask the user for a name
    namer = input("Enter a name for the person: ")

    print(f"Capturing {num_images} images for {namer}. Press 'q' to stop.")

    count = 0
    while count < num_images:
        # Read a frame from the camera
        ret, frame = cap.read()

        # Save the frame in the output folder
        img_namer = f"{namer}_{count}.png"
        img_path = os.path.join(output_folder, img_namer)
        cv2.imwrite(img_path, frame)

        print(f"Image {count + 1} captured.")
        count += 1

        # Display the frame
        cv2.imshow('Capture Images', frame)

        # Wait for 1 second
        time.sleep(0.5)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close the window
    cap.release()
    cv2.destroyAllWindows()

# Set the output folder
output_folder = r"S:\Datasets\persons"

# Capture 10 images with a 1-second delay
capture_images(output_folder, num_images=100)


Capturing 100 images for isha. Press 'q' to stop.
Image 1 captured.
Image 2 captured.
Image 3 captured.
Image 4 captured.
Image 5 captured.
Image 6 captured.
Image 7 captured.
Image 8 captured.
Image 9 captured.
Image 10 captured.
Image 11 captured.
Image 12 captured.
Image 13 captured.
Image 14 captured.
Image 15 captured.
Image 16 captured.
Image 17 captured.
Image 18 captured.
Image 19 captured.
Image 20 captured.
Image 21 captured.
Image 22 captured.
Image 23 captured.
Image 24 captured.
Image 25 captured.
Image 26 captured.
Image 27 captured.
Image 28 captured.
Image 29 captured.
Image 30 captured.
Image 31 captured.
Image 32 captured.
Image 33 captured.
Image 34 captured.
Image 35 captured.
Image 36 captured.
Image 37 captured.
Image 38 captured.
Image 39 captured.
Image 40 captured.
Image 41 captured.
Image 42 captured.
Image 43 captured.
Image 44 captured.
Image 45 captured.
Image 46 captured.
Image 47 captured.
Image 48 captured.
Image 49 captured.
Image 50 captured.
Image 51 

In [4]:
import face_recognition
import cv2
import os

known_encodings = []
known_namers = []
known_dir = r"S:\Datasets\persons"  # Change this to the absolute path of your known persons directory

# Load known persons
for file in os.listdir(known_dir):
    file_path = os.path.join(known_dir, file)
    if os.path.isfile(file_path):  # Check if the path is a file, not a directory
        img = face_recognition.load_image_file(file_path)
        # Find face encodings only if at least one face is detected
        face_encodings = face_recognition.face_encodings(img)
        if face_encodings:
            img_enc = face_encodings[0]
            known_encodings.append(img_enc)
            known_namers.append(file.split('.')[0])
        else:
            pass

In [2]:
# Import the timedelta class
from datetime import datetime, timedelta

# Initialize a variable to keep track of the last detection time (set to a time in the past)
last_detection_time = datetime.now() - timedelta(seconds=15)  # Assuming no detection in the last 15 seconds
k = 10

# Function to perform face recognition on a given frame
def recognize_faces(frame):
    global k
    global last_detection_time  # Use the global variable

    # Find face locations and encodings in the current frame
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    # Get the current timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Process each face in the frame
    person_detected = False  # Flag to check if a person is detected in the current frame
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Compare face encoding with known encodings
        results = face_recognition.compare_faces(known_encodings, face_encoding, tolerance=0.4)
        namer = "Unknown"  # Default namer if no match is found

        for i in range(len(results)):
            if results[i]:
                # Extract a more general label using split
                namer = known_namers[i].split('_')[0]
                # Update the last detection time when a person is detected
                last_detection_time = datetime.now()
                person_detected = True
                break

        # Draw rectangle and label on the frame
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, namer, (left + 2, bottom + 20), font, 0.5, (255, 255, 255), 1)

    # Add timestamp to the corner of the video
    cv2.putText(frame, timestamp, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # Check if a person is not detected in the current frame
    if not person_detected and datetime.now() - last_detection_time > timedelta(seconds=10):
        print("Person has not been detected for the last", k, " seconds.")
        k += 10
        # Reset the last detection time
        last_detection_time = datetime.now()

    # Reset k to 0 if a person is detected
    if person_detected:
        k = 10

    return frame


In [3]:
# Initialize webcam
video_capture = cv2.VideoCapture(0)  # 0 corresponds to the default camera
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    # Perform face recognition on the frame
    recognize_faces(frame)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close OpenCV windowsqq
video_capture.release()
cv2.destroyAllWindows()

NameError: name 'face_recognition' is not defined